In [1]:
import pandas as pd
import time
from selenium.common.exceptions import NoSuchElementException
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import os

In [8]:
#selenium 설정 
driver = webdriver.Chrome()
time.sleep(1)
driver.get('https://chemp.me.go.kr/sym/mnu/mpm/EgovMainMenuHome.do')

In [3]:
#로그인 액션
chem_id = driver.find_element(By.XPATH, '//*[@id="id"]')
chem_pw = driver.find_element(By.XPATH, '//*[@id="password"]')

user = 'neurolines'
pw = 're6cs3gx'

chem_id.send_keys(user)
chem_pw.send_keys(pw)

driver.find_element(By.XPATH, '//*[@id="otpNumberButton"]').click()

In [4]:
#팝업삭제
driver.switch_to.window(driver.window_handles[1])
driver.find_element(By.XPATH, '//*[@id="chkbox"]').click()
driver.switch_to.window(driver.window_handles[0])

In [5]:
#살생물제 배너 및 승인 신청 클릭 
driver.get('https://chemp.me.go.kr/cpms/bcp/BcpApplList.do')

In [6]:
#드롭다운 메뉴 가져오는 함수 
def get_selected_option(xpath):
    try:
        select_element = driver.find_element(By.XPATH, xpath)

        # Select 객체 생성
        select = Select(select_element)

        # 선택된 옵션의 텍스트 가져오기
        selected_option = select.first_selected_option.text
        # 특정 문자열 제거
        if selected_option == "=선택=":
            return ""  # 또는 None, 필요에 따라 조정 가능
        
        return selected_option
    except NoSuchElementException as e:
        print(f"Error retrieving input value: {e}")
        return NoSuchElementException


#체크 메뉴 가져오는 함수
def get_checked_checkboxes(xpath):
    checked_items = []

    # 체크 박스가 포함된 모든 <tr> 요소를 찾기
    rows = driver.find_elements(By.XPATH, xpath + '/tr')

    for row in rows:
        try:
            # 체크 박스 요소 찾기
            checkbox = row.find_element(By.TAG_NAME, 'input')
            # 체크된 경우
            if checkbox.is_selected():
                # 체크 박스가 체크된 경우 해당 항목의 텍스트 가져오기
                label_text = row.find_element(By.XPATH, './td[2]').text.strip()  # 체크 박스 옆의 텍스트
                checked_items.append(label_text)
        except Exception as e:
            print(f"Error retrieving checkbox in row: {e}")

    return checked_items


def get_input_value(xpath):
    try:
        # 해당 XPATH의 입력 요소 찾기
        input_value = driver.find_element(By.XPATH, xpath).get_attribute('value')
        return input_value
    except Exception as e:
        print(f"Error retrieving input value: {e}")
        return None


def is_checked(xpath):
    checked_items = []

    # 체크된 체크박스 찾기
    try:
        checked_checkbox = driver.find_element(By.XPATH, xpath + '//input[@type="checkbox" and @value="Y"]')
        # 체크된 체크박스의 값 가져오기
        checked_value = checked_checkbox.get_attribute('value')
        checked_items.append(checked_value)
    except Exception as e:
        print(f"Error retrieving checked checkbox: {e}")

    return checked_items


def click(xpath):
    return driver.find_element(By.XPATH, xpath).click()

In [7]:
for i in range(1, 100):
    try:
        list_tag = '//*[@id="listTb"]/tbody/tr[{0}]/td[7]'.format(i)
        #제품명 설정
        product_name = driver.find_element(By.XPATH, list_tag).text
        driver.find_element(By.XPATH, list_tag).click()
        print(i)
        i += 1
        print(product_name)
        time.sleep(2)
        driver.find_element(By.XPATH, '//*[@id="content"]/div/div[12]/table/tbody/tr[5]/td/button').click()
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(6)

        #1.1 요약 및 결론
        driver.find_element(By.XPATH, '//*[@id="bpdNavItemTree_2_span"]').click()

        driver.switch_to.frame('contentFrame')
        time.sleep(2)
        #요약 및 결론
        result = driver.find_element(By.XPATH,
                                     '//*[@id="dataSource"]/div/div/table/tbody/tr[2]/td/textarea').get_attribute(
            'value')

        description_result_data = {
            '제품명': product_name,
            '요약 및 결론': result
        }

        description_result_df = pd.DataFrame([description_result_data])
        driver.switch_to.default_content()

        # 1.2 제품 평가 요약 정보
        driver.find_element(By.XPATH, '//*[@id="bpdNavItemTree_3_span"]').click()
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        #제품 관리 정보
        product_info = driver.find_element(By.XPATH,
                                           '//*[@id="etc"]/div/div/table/tbody/tr[2]/td/textarea').get_attribute(
            'value')

        product_find_info = driver.find_element(By.XPATH,
                                                '//*[@id="etc"]/div/div/table/tbody/tr[3]/td/textarea').get_attribute(
            'value')
        divide_packaging = driver.find_element(By.XPATH,
                                               '//*[@id="etc"]/div/div/table/tbody/tr[4]/td/textarea').get_attribute(
            'value')
        confirmation_criteria_confirm_evaluation = driver.find_element(By.XPATH,
                                                                       '//*[@id="etc"]/div/div/table/tbody/tr[5]/td/textarea').get_attribute(
            'value')

        product_evaluation_info_data = {
            '제품명': product_name,
            '제품 관리 정보': product_info,
            '제품 식별 정보': product_find_info,
            '분류 표시 및 포장': divide_packaging,
            '승인 기준 준수 여부 및 승인 유효기간 평가': confirmation_criteria_confirm_evaluation
        }
        product_evaluation_info_df = pd.DataFrame([product_evaluation_info_data])
        driver.switch_to.default_content()

        #2.1.1.1 제품 관리 정보
        driver.find_element(By.XPATH, '//*[@id="bpdNavItemTree_7_span"]').click()
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        product_type = get_selected_option('//*[@id="etc"]/div/div/table/tbody/tr[3]/td/select')  #제품유형
        #제품명
        product_info_product_name = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[4]/td/textarea')
        #상품명 또는 국외에서 사용되는 상품평
        trade_name = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[5]/td/textarea')
        #개발 코드 및 번호
        develop_code = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[6]/td/input')
        #용도
        uses = get_checked_checkboxes('//*[@id="etc"]/div/div/table/tbody/tr[7]/td/div/table/tbody')

        #상세용도
        uses_detail = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[8]/td/textarea')

        # 제형 구분
        formulation_classification = get_selected_option('//*[@id="etc"]/div/div/table/tbody/tr[11]/td/select')

        # 제형 분류
        formulation_type = get_selected_option('//*[@id="etc"]/div/div/table/tbody/tr[12]/td/select')

        #OECD 화학물질 제형 분류
        oecd_formulation_type = get_selected_option('//*[@id="etc"]/div/div/table/tbody/tr[15]/td/select')

        # 제형 적용 방법
        formulation_application_method = get_selected_option('//*[@id="etc"]/div/div/table/tbody/tr[17]/td/select')

        # 적용률
        application_rate = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[19]/td/input')

        # 기타
        other = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[20]/td/input')

        # 고분자 특성
        polymer_properties = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[21]/td/div/input[1]')

        product_managing_info_data = {
            '제품명': product_name,
            '제품유형': product_type,
            '제품관리정보_제품명': product_info_product_name,
            '상품명 또는 국외에서 사용되는 상품평': trade_name,
            '개발코드 및 번호': develop_code,
            '용도': uses,
            '상세용도': uses_detail,
            '제형구분': formulation_classification,
            '제형분류': formulation_type,
            'OECD 화학물질 제형 분류': oecd_formulation_type,
            '제형 적용 방법': formulation_application_method,
            '적용률': application_rate,
            '기타': other,
            '고분자 특성 자료': polymer_properties
        }
        product_managing_info_df = pd.DataFrame([product_managing_info_data])
        driver.switch_to.default_content()

        #2.1.1.2.2 살생물 제품 구성 성분 정보
        driver.find_element(By.XPATH, '//*[@id="bpdNavItemTree_10_span"]').click()
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        #살생물제품의 개별 구성성분 정보(혼합 전) 자료명
        biocidal_product_individual_components_info_before_mixing_data_name = get_input_value(
            '//*[@id="etc"]/div/div/table/tbody/tr[4]/td/input')

        #살생물제품의 개별 구성성분 정보(혼합 전) 자료파일명
        biocidal_product_individual_components_info_before_mixing_data_file = get_input_value(
            '//*[@id="etc"]/div/div/table/tbody/tr[5]/td/div/input[1]')

        #살생물제품 최종 구성성분 정보 자료명
        biocidal_product_individual_components_info_final_data_name = get_input_value(
            '//*[@id="etc0"]/div/div/table/tbody/tr[3]/td/input')
        #살생물제품 최종 구성성분 정보 자료파일명
        biocidal_product_individual_components_info_final_data_file = get_input_value(
            '//*[@id="etc0"]/div/div/table/tbody/tr[4]/td/div/input[1]')

        # 살생물제품의 개별 구성성분 정보 데이터
        biocidal_product_ingredient_info_data = {
            '제품명': product_name,
            '살생물제품의 개별 구성성분 정보 자료명': biocidal_product_individual_components_info_before_mixing_data_name,
            '자료파일명': biocidal_product_individual_components_info_before_mixing_data_file,
            '최종 구성성분 정보 자료명': biocidal_product_individual_components_info_final_data_name,
            '최종 구성성분 정보 자료파일명': biocidal_product_individual_components_info_final_data_file
        }

        # 데이터프레임 생성
        biocidal_product_ingredient_info_df = pd.DataFrame([biocidal_product_ingredient_info_data])
        driver.switch_to.default_content()

        #2.1.1.5 제품 개발 경위
        driver.find_element(By.XPATH, '//*[@id="bpdNavItemTree_16_span"]').click()
        driver.switch_to.frame('contentFrame')
        time.sleep(2)
        develop_result = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[2]/td/textarea')

        product_develop_result_data = {
            '제품명': product_name,
            '제품 개발 경위 및 경과': develop_result
        }
        product_develop_result_df = pd.DataFrame([product_develop_result_data])
        driver.switch_to.default_content()

        #2.1.3.1.2 건강 유해성
        driver.find_element(By.XPATH, '//*[@id="bpdNavItemTree_62_span"]').click()
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        cols = ['유행성 항목', '구분', '구분근거']
        Health_Hazard = pd.DataFrame(columns=cols)

        a = 1
        b = 3
        c = 5

        for k in range(1, 77):
            a_tag = '//*[@id="etc"]/div[1]/div/table/tbody/tr[{0}]/td/select'.format(a)
            b_tag = '//*[@id="etc"]/div[1]/div/table/tbody/tr[{0}]/td/select'.format(b)
            c_tag = '//*[@id="etc"]/div[1]/div/table/tbody/tr[{0}]/td/textarea'.format(c)

            # 선택된 옵션과 입력값 가져오기
            selected_option_a = get_selected_option(a_tag)
            selected_option_b = get_selected_option(b_tag)
            input_value_c = get_input_value(c_tag)

            # 새로운 행 데이터 생성
            new_data = pd.DataFrame({
                '제품명': product_name,
                '유행성 항목': [selected_option_a],
                '구분': [selected_option_b],
                '구분근거': [input_value_c]
            })

            # DataFrame에 새로운 행 추가
            Health_Hazard_df = pd.concat([Health_Hazard, new_data], ignore_index=True)

            a += 6
            b += 6
            c += 6

            if c == 77:
                break

        driver.switch_to.default_content()

        #2.1.3.5 안전용기 또는 포장 기준 준수 정보
        driver.find_element(By.XPATH, '//*[@id="bpdNavItemTree_67_span"]').click()
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        #겉모양 
        shape = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[2]/td/input')

        # 강도 및 누수 정보
        strength_and_leakage_information = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[3]/td/input')
        #강도 및 누수 정보 첨부자료  
        strength_and_leakage_information_file = get_input_value(
            '//*[@id="etc"]/div/div/table/tbody/tr[4]/td/div/input[1]')
        # 중량·용량 정보
        weight_and_capacity_information = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[5]/td/input')
        #중량 용량 정보 첨부자료
        weight_and_capacity_information_file = get_input_value(
            '//*[@id="etc"]/div/div/table/tbody/tr[6]/td/div/input[1]')

        # 어린이 보호 포장 적용 대상 여부
        is_child_resistant_packaging_applied = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[7]/td/input')

        # 어린이 보호 포장 안전기준 적합 여부
        is_child_resistant_packaging_standard_compliant = get_input_value(
            '//*[@id="etc"]/div/div/table/tbody/tr[8]/td/input')

        # 데이터프레임 생성
        information_on_compliance_with_safety_container_or_packaging_standards_data = {
            '제품명': product_name,
            '겉모양': shape,
            '강도 및 누수 정보': strength_and_leakage_information,
            '강도 및 누수 정보 첨부자료': strength_and_leakage_information_file,
            '중량·용량 정보': weight_and_capacity_information,
            '중량·용량 정보 첨부자료': weight_and_capacity_information_file,
            '어린이 보호 포장 적용 대상 여부': is_child_resistant_packaging_applied,
            '어린이 보호 포장 안전기준 적합 여부': is_child_resistant_packaging_standard_compliant
        }

        # 데이터프레임 생성
        packaging_standards_df = pd.DataFrame(
            [information_on_compliance_with_safety_container_or_packaging_standards_data])
        driver.switch_to.default_content()

        #2.1.3.6 제품 표시사항 준수 정보
        driver.find_element(By.XPATH, '//*[@id="bpdNavItemTree_68_span"]').click()
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        #제품 표시사항 제품명 
        product_name_2136 = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[2]/td/input')

        # 살생물제품유형
        pesticide_product_type = get_selected_option('//*[@id="etc"]/div/div/table/tbody/tr[3]/td/select')

        # 유통기한
        expiration_date = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[4]/td/input')

        # 중량용량
        weight_and_capacity = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[5]/td/input')

        # 효과효능
        effectiveness_and_efficacy = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[6]/td/input')

        # 사용대상자 및 사용 범위
        target_users_and_usage_scope = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[7]/td/input')

        # 표준사용량
        standard_usage_amount = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[8]/td/input')

        #제조자 ,주소, 연락처
        company_info = get_input_value(
            '//*[@id="etc"]/div/div/table/tbody/tr[9]/td/span/input[2]') + " " + get_input_value(
            '//*[@id="etc"]/div/div/table/tbody/tr[9]/td/span/input[3]') + " " + get_input_value(
            '//*[@id="etc"]/div/div/table/tbody/tr[9]/td/span/input[4]')

        #수입제조제품: 제조국명 및 제조 회사
        import_product = get_input_value(
            '//*[@id="etc"]/div/div/table/tbody/tr[10]/td/span/input[2]') + " " + get_input_value(
            '//*[@id="etc"]/div/div/table/tbody/tr[10]/td/span/input[3]')

        #수입제조제품 : 수입자, 주소, 연락처
        import_product_company_info = get_input_value(
            '//*[@id="etc"]/div/div/table/tbody/tr[11]/td/span/input[2]') + " " + get_input_value(
            '//*[@id="etc"]/div/div/table/tbody/tr[11]/td/span/input[3]') + " " + get_input_value(
            '//*[@id="etc"]/div/div/table/tbody/tr[11]/td/span/input[4]')

        # 어린이 보호표장의 표시
        child_resistant_packaging_labeling = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[12]/td/input')

        # 살생물물질
        pesticidal_substance = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[13]/td/input')

        # 나노물질
        nanomaterial = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[14]/td/input')

        # 기타 화학물질
        other_chemical_substances = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[15]/td/input')

        #제품 유해성 위험성
        product_danger = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[16]/td/textarea')

        # 사용방법
        usage_instructions = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[17]/td/textarea')

        # 사용상 주의 사항
        usage_warnings = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[18]/td/textarea')

        # 승인번호
        approval_number = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[19]/td/input')

        # 제조번호
        manufacturing_number = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[20]/td/input')

        product_label_data = {
            '제품명': product_name_2136,
            '살생물제품유형': pesticide_product_type,
            '유통기한': expiration_date,
            '중량·용량': weight_and_capacity,
            '효과효능': effectiveness_and_efficacy,
            '사용대상자 및 사용 범위': target_users_and_usage_scope,
            '표준사용량': standard_usage_amount,
            '제조자, 주소, 연락처': company_info,
            '수입제조제품: 제조국명 및 제조 회사': import_product,
            '수입제조제품: 수입자, 주소, 연락처': import_product_company_info,
            '어린이 보호표장의 표시': child_resistant_packaging_labeling,
            '살생물물질': pesticidal_substance,
            '나노물질': nanomaterial,
            '기타 화학물질': other_chemical_substances,
            '제품 유해성 위험성': product_danger,
            '사용방법': usage_instructions,
            '사용상 주의 사항': usage_warnings,
            '승인번호': approval_number,
            '제조번호': manufacturing_number
        }

        product_label_df = pd.DataFrame([product_label_data])
        driver.switch_to.default_content()

        #2.1.5.1 살생물제품 용도 및 상세용도
        driver.find_element(By.XPATH, '//*[@id="bpdNavItemTree_74_span"]').click()
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        #살생제품 용도
        salsaeng_use = get_checked_checkboxes('//*[@id="etc0"]/div/div/table/tbody/tr[2]/td/div/table/tbody')
        #기타
        etc = get_input_value('//*[@id="etc0"]/div/div/table/tbody/tr[3]/td/textarea')
        #살생제품유형
        salsaeng_type = get_selected_option('//*[@id="etc0"]/div/div/table/tbody/tr[5]/td/select')

        salsaeng_data = {
            '제품명': product_name,
            '살생제품 용도': salsaeng_use,
            '기타': etc,
            '살생제품 유형': salsaeng_type
        }

        salsaeng_df = pd.DataFrame([salsaeng_data])
        driver.switch_to.default_content()

        #2.1.5.3 살생물제품 노출 정보
        click('//*[@id="bpdNavItemTree_76_span"]')
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        #살생물제품의 노출 정보 : 노출정보
        salsaeng_expose_info = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[2]/td/textarea')
        #살생물제품의 노출 정보 : 첨부자료
        salsaeng_expose_info_file = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[4]/td/div/input[1]')
        #제품 사용/폐기에 따른 인체 및 환경 노출 관련 정보: 노출정보
        product_use_expose_info = get_input_value('//*[@id="etc0"]/div/div/table/tbody/tr[2]/td/textarea')
        #제품 사용/폐기에 따른 인체 및 환경 노출 관련 정보: 첨부자료
        product_use_expose_info_file = get_input_value('//*[@id="etc0"]/div/div/table/tbody/tr[4]/td/div/input[1]')

        salsaeng_product_expose_data = {
            '제품명': product_name,
            '살생물 제품의 노출 정보': salsaeng_expose_info,
            '살생물 제품의 노출 정보 첨부자료': salsaeng_expose_info_file,
            '제품사용에 따른 인체 환경 노출 관련 정보': product_use_expose_info,
            '제품사용에 따른 인체 환경 노출 관련 정보 첨부자료': product_use_expose_info_file
        }
        salsaeng_product_expose_df = pd.DataFrame([salsaeng_product_expose_data])
        driver.switch_to.default_content()

        #2.1.6.2 사용상 주의사항
        click('//*[@id="bpdNavItemTree_79_span"]')
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        #화재 진압시에 착용할 보호구 
        fire_protection = get_input_value('//*[@id="etc"]/div[1]/div/table/tbody/tr[3]/td/textarea')
        #연소 위험성
        salsaeng_danger_point = get_input_value('//*[@id="etc"]/div[1]/div/table/tbody/tr[4]/td/textarea')

        # 소화제
        fire_extinguisher = get_input_value('//*[@id="etc"]/div[1]/div/table/tbody/tr[6]/td/textarea')
        #안전상의 이유로 사용하지 않는 소화제 
        forbidden_fire_extinguisher = get_input_value('//*[@id="etc"]/div[1]/div/table/tbody/tr[7]/td/textarea')

        #개인보호구 착용
        personal_protection = get_input_value('//*[@id="etc"]/div[2]/div/table/tbody/tr[3]/td/textarea')

        # 환경예방조치
        environmental_preventive_measures = get_input_value('//*[@id="etc"]/div[2]/div/table/tbody/tr[4]/td/textarea')

        # 방제약품
        pest_control_chemicals = get_input_value('//*[@id="etc"]/div[2]/div/table/tbody/tr[5]/td/textarea')

        # 용기 재질
        container_material = get_input_value('//*[@id="etc"]/div[3]/div/table/tbody/tr[3]/td/textarea')

        # 피해야할 물질
        substances_to_avoid = get_input_value('//*[@id="etc"]/div[3]/div/table/tbody/tr[4]/td/textarea')

        # 보관조건
        storage_conditions = get_input_value('//*[@id="etc"]/div[3]/div/table/tbody/tr[5]/td/textarea')

        # 정전기방지
        static_electricity_prevention = get_input_value('//*[@id="etc"]/div[3]/div/table/tbody/tr[6]/td/textarea')

        # 주의사항
        precautions = get_input_value('//*[@id="etc"]/div[3]/div/table/tbody/tr[7]/td/textarea')

        # 응급조치
        emergency_measures = get_input_value('//*[@id="etc"]/div[3]/div/table/tbody/tr[8]/td/textarea')

        salsaeng_expose_data = {
            '제품명': product_name,
            '화재 진압시에 착용할 보호구': fire_protection,
            '살생물물질 자체 또는 살생물물질의 연소로 부터 발생하는 위험성(유해화학물질 등)': salsaeng_danger_point,
            '적절한 소화제': fire_extinguisher,
            '안전상의 이유로 사용하지 않는 소화제': forbidden_fire_extinguisher,
            '개인보호구 착용': personal_protection,
            '환경예방조치': environmental_preventive_measures,
            '방제약품, 장비, 방법': pest_control_chemicals,
            '저장 공간 및 저장 용기에 대한 재질, 특정한 모양ㆍ형태, 환기시설 등': container_material,
            '피해야 할 물질과 조건, 유해한 분해 생성물 등': substances_to_avoid,
            '온도, 습도, 빛 등 저장ㆍ보관 조건': storage_conditions,
            '정전기 방지 조치': static_electricity_prevention,
            '사용시 주의사항': precautions,
            '사용자에 대한 응급조치 방법': emergency_measures
        }

        salsaeng_expose_df = pd.DataFrame([salsaeng_expose_data])
        driver.switch_to.default_content()

        #2.1.6.3 폐기방법
        click('//*[@id="bpdNavItemTree_80_span"]')
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        #적절한 폐기 방법 
        profit_del_method = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[2]/td/textarea')
        #폐기물 관리 대책
        managing_method = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[3]/td/textarea')
        #폐기물 관리 및 처리 규정 
        managing_compliance = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[4]/td/textarea')

        del_method_data = {
            '제품명': product_name,
            '적절한 폐기 방법': profit_del_method,
            '폐기물 관리 대책': managing_method,
            '폐기물 관리 및 처리 규정': managing_compliance
        }

        del_method_df = pd.DataFrame([del_method_data])
        driver.switch_to.default_content()

        #2.2.4.1.1 작업자 위해도 결정
        click('//*[@id="bpdNavItemTree_112_span"]')
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        #살생물제품 생산
        salsaeng_product_make = get_input_value('//*[@id="etc"]/div[1]/div/table/tbody/tr[2]/td/textarea')
        #첨부파일
        file_22411 = get_input_value('//*[@id="etc"]/div[1]/div/table/tbody/tr[3]/td/div/input[1]')

        worker_safety_decision_data = {
            '제품명': product_name,
            '살생물제품 생산': salsaeng_product_make,
            '첨부파일': file_22411
        }
        worker_safety_decision_df = pd.DataFrame([worker_safety_decision_data])
        driver.switch_to.default_content()

        #2.2.4.2.1.2 국소 영향
        click('//*[@id="bpdNavItemTree_116_span"]')
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        #제출여부
        is_submit = get_selected_option('//*[@id="dataSource"]/div/div/table/tbody/tr[2]/td/select')
        #국소영향
        gookSo_effect = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[2]/td/textarea')

        gookSo_effect_224212_data = {
            '제품명': product_name,
            '제출여부': is_submit,
            '국소영향': gookSo_effect
        }

        gookSo_effect_224212_df = pd.DataFrame([gookSo_effect_224212_data])
        driver.switch_to.default_content()

        #2.2.4.2.2.2 국소 영향
        click('//*[@id="bpdNavItemTree_119_span"]')
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        #제출여부
        is_submit = get_selected_option('//*[@id="dataSource"]/div/div/table/tbody/tr[2]/td/select')
        #국소영향
        gookSo_effect = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[2]/td/textarea')

        gookSo_effect_224222_data = {
            '제품명': product_name,
            '제출여부': is_submit,
            '국소영향': gookSo_effect
        }

        gookSo_effect_224222_df = pd.DataFrame([gookSo_effect_224222_data])
        driver.switch_to.default_content()

        #2.2.4.2.3.2 국소 영향
        click('//*[@id="bpdNavItemTree_122_span"]')
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        #제출여부
        is_submit = get_selected_option('//*[@id="dataSource"]/div/div/table/tbody/tr[2]/td/select')
        #국소영향
        gookSo_effect = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[2]/td/textarea')

        gookSo_effect_224232_data = {
            '제품명': product_name,
            '제출여부': is_submit,
            '국소영향': gookSo_effect
        }

        gookSo_effect_224232_df = pd.DataFrame([gookSo_effect_224232_data])
        driver.switch_to.default_content()

        #2.2.5.1 위해성평가 결과 요약
        click('//*[@id="bpdNavItemTree_124_a"]')
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        #결론
        result_2251 = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[2]/td/textarea')

        rist_assessment_data = {
            '제품명': product_name,
            '결론': result_2251
        }

        rist_assessment_df = pd.DataFrame([rist_assessment_data])
        driver.switch_to.default_content()

        #2.3.5.1 환경 위해성평가 결과 요약
        click('//*[@id="bpdNavItemTree_155_span"]')
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        #결론
        result_2351 = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[2]/td/textarea')

        env_rist_assessment_data = {
            '제품명': product_name,
            '결론': result_2351
        }

        env_rist_assessment_df = pd.DataFrame([env_rist_assessment_data])
        driver.switch_to.default_content()

        #2.4.1.4 제품 또는 처리제품의 실제 표시내용
        click('//*[@id="bpdNavItemTree_161_span"]')
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        #결론
        result_2414 = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[2]/td/textarea')

        product_show_detail_data = {
            '제품명': product_name,
            '결론': result_2414
        }

        product_show_detail_df = pd.DataFrame([product_show_detail_data])
        driver.switch_to.default_content()

        #3.1.1 효과ㆍ효능 평가 결과
        click('//*[@id="bpdNavItemTree_164_span"]')
        driver.switch_to.frame('contentFrame')
        time.sleep(2)
        #살생물 물질의 기능 
        salsaeng_material_func = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[2]/td/textarea')

        #작용시작 및 근거
        effect_start = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[4]/td/textarea')

        #제어 대상 생물체
        control_material = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[6]/td/textarea')

        #내성발생 가능성 및 근거 
        possibility_of_resistance_development_and_evidence = get_input_value(
            '//*[@id="etc"]/div/div/table/tbody/tr[8]/td/textarea')

        effect_result_311_data = {
            '제품명': product_name,
            '살생물물질의 기능': salsaeng_material_func,
            '작용시작 및 근거': effect_start,
            '제어대상 생물체': control_material,
            '내성발생 가능성 및 근거': possibility_of_resistance_development_and_evidence
        }

        effect_result_311_df = pd.DataFrame([effect_result_311_data])
        driver.switch_to.default_content()

        #3.1.2 위해성 평가 결과
        click('//*[@id="bpdNavItemTree_165_span"]')
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        #위해성 평가 결과
        result_312 = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[2]/td/textarea')

        rist_assessment_312_data = {
            '제품명': product_name,
            '결론': result_312
        }

        rist_assessment_312_df = pd.DataFrame([rist_assessment_312_data])
        driver.switch_to.default_content()

        #3.2.1.2.1 유해성 구분
        click('//*[@id="bpdNavItemTree_170_span"]')
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        # 급성 경구 독성
        acute_oral_toxicity = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[2]/td/textarea')

        # 급성 경피 독성
        acute_dermal_toxicity = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[4]/td/textarea')

        # 급성 흡입 독성(가스)
        acute_inhalation_toxicity_gas = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[6]/td/textarea')

        # 급성 흡입 독성(증기)
        acute_inhalation_toxicity_air = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[8]/td/textarea')

        # 특정 표적 장기 독성
        specific_target_organ_toxicity = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[10]/td/textarea')

        # 발암성
        carcinogenicity = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[12]/td/textarea')

        # 생식 세포 변이원성
        germ_cell_mutagenicity = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[14]/td/textarea')

        # 생식독성
        reproductive_toxicity = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[16]/td/textarea')

        #발암성, 변이원성, 생식독성(CMR) 결론
        crm = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[18]/td/textarea')

        # 유해성 데이터 딕셔너리 생성
        hazard_data = {
            '제품명': product_name,
            '급성 경구 독성': acute_oral_toxicity,
            '급성 경피 독성': acute_dermal_toxicity,
            '급성 흡입 독성(가스)': acute_inhalation_toxicity_gas,
            '급성 흡입 독성(증기)': acute_inhalation_toxicity_air,
            '특정 표적 장기 독성': specific_target_organ_toxicity,
            '발암성': carcinogenicity,
            '생식 세포 변이원성': germ_cell_mutagenicity,
            '생식독성': reproductive_toxicity,
            '발암성, 변이원성, 생식독성(CMR) 결론': crm
        }
        hazard_df = pd.DataFrame([hazard_data])
        driver.switch_to.default_content()

        #3.2.1.2.2 유해성 평가
        click('//*[@id="bpdNavItemTree_171_span"]')
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        # 인체에 대한 내분비계 장애영향
        endocrine_disruption_effects_on_human = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[2]/td/textarea')

        # 환경에 대한 내분비계 장애영향
        endocrine_disruption_effects_on_environment = get_input_value(
            '//*[@id="etc"]/div/div/table/tbody/tr[3]/td/textarea')

        # 내분비계 장애영향(EDCs) 결론
        endocrine_disruption_conclusion_edcs = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[4]/td/textarea')

        # 인체에 대한 발달 신경독성 또는 면역 독성영향
        developmental_neurotoxicity_or_immunotoxicity_effects_on_human = get_input_value(
            '//*[@id="etc"]/div/div/table/tbody/tr[6]/td/textarea')

        # 환경에 대한 발달 신경독성 또는 면역 독성영향
        developmental_neurotoxicity_or_immunotoxicity_effects_on_environment = get_input_value(
            '//*[@id="etc"]/div/div/table/tbody/tr[7]/td/textarea')

        # 발달 신경독성 또는 면역 독성 영향 결론
        developmental_neurotoxicity_or_immunotoxicity_conclusion = get_input_value(
            '//*[@id="etc"]/div/div/table/tbody/tr[8]/td/textarea')

        # 수은 화합물 정보
        mercury_compound_information = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[10]/td/textarea')

        #PBT VPvB 정보 
        pbt_vPvB = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[12]/td/textarea')

        # 위험성 평가 데이터 딕셔너리 생성
        rist_assessment_data_32122 = {
            '제품명': product_name,
            '인체에 대한 내분비계 장애영향': endocrine_disruption_effects_on_human,
            '환경에 대한 내분비계 장애영향': endocrine_disruption_effects_on_environment,
            '내분비계 장애영향(EDCs) 결론': endocrine_disruption_conclusion_edcs,
            '인체에 대한 발달 신경독성 또는 면역 독성영향': developmental_neurotoxicity_or_immunotoxicity_effects_on_human,
            '환경에 대한 발달 신경독성 또는 면역 독성영향': developmental_neurotoxicity_or_immunotoxicity_effects_on_environment,
            '발달 신경독성 또는 면역 독성 영향 결론': developmental_neurotoxicity_or_immunotoxicity_conclusion,
            '수은 화합물 정보': mercury_compound_information,
            'PBT VPvB 정보': pbt_vPvB
        }

        rist_assessment_df_32122 = pd.DataFrame([rist_assessment_data_32122])
        driver.switch_to.default_content()

        #3.2.1.2.3 PBT 또는 vPvB 평가
        click('//*[@id="bpdNavItemTree_172_span"]')
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        #수중(담수,기수)(>40일)
        water_40 = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[2]/td/textarea')

        #토양 ( > 120일 )
        land_120 = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[3]/td/textarea')

        #퇴적물 (담수, 기수) > 120일
        sediment_120 = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[4]/td/textarea')

        #수중(담수, 기수) ( > 60일 )
        vp_water_60 = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[6]/td/textarea')

        #토양 ( > 180일 )
        vp_land_180 = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[7]/td/textarea')

        #퇴적물 (담수, 기수) > 180일
        sediment_180 = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[8]/td/textarea')

        #잔류성 / 고잔류성 (P/vP) 결론
        vp_result = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[9]/td/textarea')

        #생물축적성(B) / 생물농축계수(BCF) > 2000
        bcf_2000 = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[11]/td/textarea')

        #고생물축적성(vB) / 생물농축계수(BCF) >5000
        vb_bcf_5000 = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[12]/td/textarea')

        #축적성/고축적성(B/vB) 결론
        vb_result = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[13]/td/textarea')

        #어류, 물벼룩류 또는 담소조류에 대한 만성독성
        fish_poison = get_input_value('//*[@id="etc0"]/div[1]/div/table/tbody/tr[2]/td/textarea')

        #반복투여(노출)독성/구분1
        repeat_injection_poison_type1 = get_input_value('//*[@id="etc0"]/div[1]/div/table/tbody/tr[3]/td/textarea')

        #반복투여(노출)독성/구분2
        repeat_injection_poison_type2 = get_input_value('//*[@id="etc0"]/div[1]/div/table/tbody/tr[4]/td/textarea')

        # 변이원성/구분1A
        mutagenicity_type_1A = get_input_value('//*[@id="etc0"]/div[1]/div/table/tbody/tr[5]/td/textarea')

        # 변이원성/구분1B
        mutagenicity_type_1B = get_input_value('//*[@id="etc0"]/div[1]/div/table/tbody/tr[6]/td/textarea')

        # 발암성/구분1A
        carcinogenicity_type_1A = get_input_value('//*[@id="etc0"]/div[1]/div/table/tbody/tr[7]/td/textarea')

        # 발암성/구분1B
        carcinogenicity_type_1B = get_input_value('//*[@id="etc0"]/div[1]/div/table/tbody/tr[8]/td/textarea')

        # 생식독성/구분1A
        reproductive_toxicity_1A = get_input_value('//*[@id="etc0"]/div[1]/div/table/tbody/tr[9]/td/textarea')

        # 생식독성/구분1B
        reproductive_toxicity_1B = get_input_value('//*[@id="etc0"]/div[1]/div/table/tbody/tr[10]/td/textarea')

        #생식독성/구분2
        reproductive_toxicity_2 = get_input_value('//*[@id="etc0"]/div[1]/div/table/tbody/tr[11]/td/textarea')

        #독성에 대한 결론 
        posion_result = get_input_value('//*[@id="etc0"]/div[1]/div/table/tbody/tr[12]/td/textarea')

        pbt_vPvB_32123_data = {
            '제품명': product_name,
            '수중(담수, 기수)(>40일)': water_40,
            '토양(> 120일)': land_120,
            '퇴적물(담수, 기수) > 120일': sediment_120,
            '수중(담수, 기수)(> 60일)': vp_water_60,
            '토양(> 180일)': vp_land_180,
            '퇴적물(담수, 기수) > 180일': sediment_180,
            '잔류성/고잔류성(P/vP) 결론': vp_result,
            '생물축적성(B) / 생물농축계수(BCF) > 2000': bcf_2000,
            '고생물축적성(vB) / 생물농축계수(BCF) > 5000': vb_bcf_5000,
            '축적성/고축적성(B/vB) 결론': vb_result,
            '어류, 물벼룩류 또는 담소조류에 대한 만성독성': fish_poison,
            '반복투여(노출)독성/구분1': repeat_injection_poison_type1,
            '반복투여(노출)독성/구분2': repeat_injection_poison_type2,
            '변이원성/구분1A': mutagenicity_type_1A,
            '변이원성/구분1B': mutagenicity_type_1B,
            '발암성/구분1A': carcinogenicity_type_1A,
            '발암성/구분1B': carcinogenicity_type_1B,
            '생식독성/구분1A': reproductive_toxicity_1A,
            '생식독성/구분1B': reproductive_toxicity_1B,
            '생식독성/구분2': reproductive_toxicity_2,
            '독성에 대한 결론': posion_result
        }

        pbt_vPvB_32123_df = pd.DataFrame([pbt_vPvB_32123_data])
        driver.switch_to.default_content()

        #3.2.1.3 위해성평가 기준
        click('//*[@id="bpdNavItemTree_173_span"]')
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        #비발암위해도-노출한계(MOE)가 100이하인 경우
        moe_100 = get_input_value('//*[@id="etc"]/div[1]/div/table/tbody/tr[2]/td/textarea')
        #비발암위해도-유해지수(HQ)가 1이상인 경우
        hq_1 = get_input_value('//*[@id="etc"]/div[1]/div/table/tbody/tr[3]/td/textarea')
        #비말암위해도-그 외 위해도 평가
        etc_1 = get_input_value('//*[@id="etc"]/div[1]/div/table/tbody/tr[4]/td/textarea')

        #비역치를 가정한 발암위해도-평가발암위해도의 경우 노출한계(MOE)가 10,000이하인 경우
        moe_10000 = get_input_value('//*[@id="etc"]/div[1]/div/table/tbody/tr[6]/td/textarea')
        #비역치를 가정한 발암위해도-초과발암위해도가 10-⁴이상인 경우
        moe_2_10_4 = get_input_value('//*[@id="etc"]/div[1]/div/table/tbody/tr[7]/td/textarea')
        #비역치를 가정한 발암위해도-그 외 위해도 평가
        etc_2_10 = get_input_value('//*[@id="etc"]/div[1]/div/table/tbody/tr[8]/td/textarea')

        #정량적 생태위해도를 유해지수(HQ)로나타낸 경우, 유해지수가 1보다 클 경우(HQ>1)
        hq_1_2 = get_input_value('//*[@id="etc0"]/div/div/table/tbody/tr[2]/td/textarea')
        #환경에 대한 위해성평가 기준 - 그 외 환경에 대한 위해도 평가
        env_report = get_input_value('//*[@id="etc0"]/div/div/table/tbody/tr[3]/td/textarea')
        #환경에 대한 위해성평가 결론
        env_result_3213 = get_input_value('//*[@id="etc0"]/div/div/table/tbody/tr[4]/td/textarea')

        risk_assessment_criteria_data_3213 = {

            '제품명': product_name,
            '비발암위해도-노출한계(MOE) 100 이하': moe_100,
            '비발암위해도-유해지수(HQ) 1 이상': hq_1,
            '비발암위해도-그 외 위해도 평가': etc_1,
            '비역치를 가정한 발암위해도-평가발암위해도의 경우 노출한계(MOE) 10,000 이하': moe_10000,
            '비역치를 가정한 발암위해도-초과발암위해도가 10^-4 이상': moe_2_10_4,
            '비역치를 가정한 발암위해도-그 외 위해도 평가': etc_2_10,
            '정량적 생태위해도를 유해지수(HQ)로 나타낸 경우(HQ > 1)': hq_1_2,
            '환경에 대한 위해성평가 기준 - 그 외 환경에 대한 위해도 평가': env_report,
            '환경에 대한 위해성평가 결론': env_result_3213,

        }

        rist_risk_assessment_criteria_3213_df = pd.DataFrame([risk_assessment_criteria_data_3213])
        driver.switch_to.default_content()

        #3.2.1.4 유해생물 제거 등 효과ㆍ효능 평가 기준
        click('//*[@id="bpdNavItemTree_174_span"]')
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        #살생물제품의 분류·표시 및 포장에 나타낸 효과·효능에 관한 정보와 시험결과 일치 여부
        is_test_correct = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[2]/td/textarea')
        #표준사용량, 사용방법의 조건에서 사용목적과 용도에 적합한 효과효능 기준 충족 여부
        standard_usage_amount_method_3214 = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[4]/td/textarea')
        #유해생물 제거 등의 효과ㆍ효능 평가 결론
        effect_efficient_result = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[6]/td/textarea')

        danger_animal_del_effect_data = {
            '제품명': product_name,
            '살생물제품의 분류·표시 및 포장에 나타낸 효과·효능에 관한 정보와 시험결과 일치 여부': is_test_correct,
            '표준사용량, 사용방법의 조건에서 사용목적과 용도에 적합한 효과효능 기준 충족 여부': standard_usage_amount_method_3214,
            '유해생물 제거 등의 효과ㆍ효능 평가 결론': effect_efficient_result,
        }

        danger_animal_del_effect_df = pd.DataFrame([danger_animal_del_effect_data])
        driver.switch_to.default_content()

        #3.2.1.5 유해생물에 대한 내성 평가
        click('//*[@id="bpdNavItemTree_175_span"]')
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        #살생물제품의 표적생물체에 대한 내성 또는 교차내성 확인 여부
        is_check = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[2]/td/textarea')

        #살생물제품이 표적생물체에 대한 내성 또는 교차내성을 나타낼 가능성이 있는 경우, 적절한 저감 방안 제시 여부
        is_cross_check_method = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[4]/td/textarea')

        #유해생물에 대한 내성 평가 결론
        result_3215 = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[6]/td/textarea')

        naesung_report_3215 = {
            '제품명': product_name,
            '살생물제품의 표적생물체에 대한 내성 또는 교차내성 확인 여부': is_check,
            '살생물제품이 표적생물체에 대한 내성 또는 교차내성을 나타낼 가능성이 있는 경우, 적절한 저감 방안 제시 여부': is_cross_check_method,
            '유해생물에 대한 내성 평가 결론': result_3215
        }

        naesung_report_3215_df = pd.DataFrame([naesung_report_3215])
        driver.switch_to.default_content()

        #3.2.1.6 척추동물에 대한 영향 평가
        click('//*[@id="bpdNavItemTree_176_span"]')
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        #제거 등을 목적으로 하는 척추동물이 지속적인 고통없이 의식이 사람짐과 동시에 처리되는 경우
        concurrent_end = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[2]/td/textarea')

        #살생물제품 사용 즉시 신속하게 처리되는 경우
        immediatly_end = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[4]/td/textarea')

        #명백한 고통 없이 생체 기능이 서서히 저하되는 경우
        no_pain = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[6]/td/textarea')

        #척추동물에 대한 승인 평가 결론
        result_3216 = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[8]/td/textarea')

        animal_report_3216_data = {
            '제품명': product_name,
            '제거 등을 목적으로 하는 척추동물이 지속적인 고통없이 의식이 사람짐과 동시에 처리되는 경우': concurrent_end,
            '살생물제품 사용 즉시 신속하게 처리되는 경우': immediatly_end,
            '명백한 고통 없이 생체 기능이 서서히 저하되는 경우': no_pain,
            '척추동물에 대한 승인 평가 결론': result_3216
        }
        animal_report_3216_df = pd.DataFrame([animal_report_3216_data])
        driver.switch_to.default_content()

        #3.2.1.7 안전용기 또는 포장에 대한 평가

        click('//*[@id="bpdNavItemTree_177_span"]')
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        #안전용기 또는 포장 사용 여부
        is_use_3217 = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[2]/td/textarea')

        packaging_evaluation_data = {
            '제품명': product_name,
            '안전용기 또는 포장 사용 여부': is_use_3217
        }

        packaging_evaluation_df = pd.DataFrame([packaging_evaluation_data])
        driver.switch_to.default_content()

        #3.2.2 승인 유효기간 평가
        click('//*[@id="bpdNavItemTree_178_span"]')
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        #기준
        standard_322 = get_checked_checkboxes('//*[@id="etc"]/div/div/table/tbody/tr[5]/td/div/table/tbody')

        #유효기간
        expiration_date_322 = get_selected_option('//*[@id="etc"]/div/div/table/tbody/tr[6]/td/select')

        #평가
        evulation_322 = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[7]/td/textarea')

        #승인기준에 따른 유효기간 평가 결론
        evulation_result_322 = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[8]/td/textarea')

        #승인 대상 살생물제품이 산업용으로만 사용함에 대한 타당성(법 제20조제4항제1호) - 첨부자료
        product_info_file_322 = get_input_value('//*[@id="etc0"]/div/div/table/tbody/tr[7]/td/div/input[1]')

        #승인 대상 제품을 대체할 수 있는 위해성이 낮은 제품이 없어 해당 살생물제품의 사용이 국민의 건강 및 환경에 필요함에 대한 타당성(법 제20조제4항제2호) - 첨부자료 
        product_info_file_322_2 = get_input_value('//*[@id="etc0"]/div/div/table/tbody/tr[10]/td/div/input[1]')

        expiration_date_evulation_322_data = {
            '제품명': product_name,
            '기준': standard_322,
            '유효기간': expiration_date_322,
            '평가': evulation_322,
            '승인기준에 따른 유효기간 평가 결론': evulation_result_322,
            '승인 대상 살생물제품이 산업용으로만 사용함에 대한 타당성(법 제20조제4항제1호) - 첨부자료': product_info_file_322,
            '승인 대상 제품을 대체할 수 있는 위해성이 낮은 제품이 없어 해당 살생물제품의 사용이 국민의 건강 및 환경에 필요함에 대한 타당성(법 제20조제4항제2호) - 첨부자료': product_info_file_322_2,
        }

        expiration_date_evulation_322_df = pd.DataFrame([expiration_date_evulation_322_data])
        driver.switch_to.default_content()

        #3.3.1 위해저감 방안
        click('//*[@id="bpdNavItemTree_180_span"]')
        driver.switch_to.frame('contentFrame')
        time.sleep(2)

        #위해저감 방안
        method_331 = get_input_value('//*[@id="etc"]/div/div/table/tbody/tr[2]/td/textarea')

        method_331_data = {
            '제품명': product_name,
            '위해저감 방안': method_331
        }

        method_331_df = pd.DataFrame([method_331_data])
        driver.switch_to.default_content()

        #4.1 종말점 정보
        click('//*[@id="bpdNavItemTree_182_span"]')
        driver.switch_to.frame('contentFrame')
        time.sleep(2)
        
        #종말점 정보
        end_point_info = get_input_value('//*[@id="collabo"]/div/div/table/tbody/tr[1]/td/textarea')
        #첨부자료
        end_point_file = get_input_value('//*[@id="collabo"]/div/div/table/tbody/tr[3]/td/div/input[1]')

        end_point_data = {
            '제품명': product_name,
            '종말점 정보': end_point_info,
            '첨부자료': end_point_file
        }
        end_point_df = pd.DataFrame([end_point_data])
        driver.switch_to.default_content()

        #저장
        save_path = r"C:\Users\neurolines\Documents\chemp\살생물 제품 안정성 종합자료"
        file_name = f"{product_name} 살생물 제품 안전성에 관한 종합 자료.xlsx"
        full_path = os.path.join(save_path, file_name)

        with pd.ExcelWriter(full_path) as writer:
            description_result_df.to_excel(writer, sheet_name='1.1 요약 및 결론', index=False)
            product_evaluation_info_df.to_excel(writer, sheet_name='1.2 제품 평가 요약 정보', index=False)
            product_managing_info_df.to_excel(writer, sheet_name='2.1.1.1 제품 관리 정보', index=False)
            biocidal_product_ingredient_info_df.to_excel(writer, sheet_name='2.1.1.2.2 살생물 제품 구성 성분 정보', index=False)
            product_develop_result_df.to_excel(writer, sheet_name='2.1.1.5 제품 개발 경위', index=False)
            Health_Hazard_df.to_excel(writer, sheet_name='2.1.3.1.2 건강 유해성', index=False)
            packaging_standards_df.to_excel(writer, sheet_name='2.1.3.5 안전용기 또는 포장 기준 준수 정보', index=False)
            product_label_df.to_excel(writer, sheet_name='2.1.3.6 제품 표시사항 준수 정보', index=False)
            salsaeng_df.to_excel(writer, sheet_name='2.1.5.1 살생물제품 용도 및 상세용도', index=False)
            salsaeng_product_expose_df.to_excel(writer, sheet_name='2.1.5.3 살생물제품 노출 정보', index=False)
            salsaeng_expose_df.to_excel(writer, sheet_name='2.1.6.2 사용상 주의사항', index=False)
            del_method_df.to_excel(writer, sheet_name='2.1.6.3 폐기방법', index=False)
            worker_safety_decision_df.to_excel(writer, sheet_name='2.2.4.1.1 작업자 위해도 결정', index=False)
            gookSo_effect_224212_df.to_excel(writer, sheet_name='2.2.4.2.1.2 국소 영향', index=False)
            gookSo_effect_224222_df.to_excel(writer, sheet_name='2.2.4.2.2.2 국소 영향', index=False)
            gookSo_effect_224232_df.to_excel(writer, sheet_name='2.2.4.2.3.2 국소 영향', index=False)
            rist_assessment_df.to_excel(writer, sheet_name='2.2.5.1 위해성평가 결과 요약', index=False)
            env_rist_assessment_df.to_excel(writer, sheet_name='2.3.5.1 환경 위해성평가 결과 요약', index=False)
            product_show_detail_df.to_excel(writer, sheet_name='2.4.1.4 제품 또는 처리제품의 실제 표시내용', index=False)
            effect_result_311_df.to_excel(writer, sheet_name='3.1.1 효과ㆍ효능 평가 결과', index=False)
            rist_assessment_312_df.to_excel(writer, sheet_name='3.1.2 위해성 평가 결과', index=False)
            hazard_df.to_excel(writer, sheet_name='3.2.1.2.1 유해성 구분', index=False)
            rist_assessment_df_32122.to_excel(writer, sheet_name='3.2.1.2.2 유해성 평가', index=False)
            pbt_vPvB_32123_df.to_excel(writer, sheet_name='3.2.1.2.3 PBT 또는 vPvB 평가', index=False)
            rist_risk_assessment_criteria_3213_df.to_excel(writer, sheet_name='3.2.1.3 위해성평가 기준', index=False)
            danger_animal_del_effect_df.to_excel(writer, sheet_name='3.2.1.4 유해생물 제거 등 효과ㆍ효능 평가 기준', index=False)
            naesung_report_3215_df.to_excel(writer, sheet_name='3.2.1.5 유해생물에 대한 내성 평가', index=False)
            animal_report_3216_df.to_excel(writer, sheet_name='3.2.1.6 척추동물에 대한 영향 평가', index=False)
            packaging_evaluation_df.to_excel(writer, sheet_name='3.2.1.7 안전용기 또는 포장에 대한 평가', index=False)
            expiration_date_evulation_322_df.to_excel(writer, sheet_name='3.2.2 승인 유효기간 평가', index=False)
            method_331_df.to_excel(writer, sheet_name='3.3.1 위해저감 방안', index=False)
            end_point_df.to_excel(writer, sheet_name='4.1 종말점 정보', index=False)

        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        driver.back()
        print(i)

    except NoSuchElementException:
        print("no_such_발생!")
        break

1
제품명14
2
2
제품명13
Error retrieving input value: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=130.0.6723.59)
Stacktrace:
	GetHandleVerifier [0x00007FF74E6E3AB5+28005]
	(No symbol) [0x00007FF74E6483B0]
	(No symbol) [0x00007FF74E4E580A]
	(No symbol) [0x00007FF74E4BFA85]
	(No symbol) [0x00007FF74E562AD7]
	(No symbol) [0x00007FF74E57B1B1]
	(No symbol) [0x00007FF74E55B7E3]
	(No symbol) [0x00007FF74E5275C8]
	(No symbol) [0x00007FF74E528731]
	GetHandleVerifier [0x00007FF74E9D643D+3118829]
	GetHandleVerifier [0x00007FF74EA26C90+3448640]
	GetHandleVerifier [0x00007FF74EA1CF0D+3408317]
	GetHandleVerifier [0x00007FF74E7AA40B+841403]
	(No symbol) [0x00007FF74E65340F]
	(No symbol) [0x00007FF74E64F484]
	(No symbol) [0x00007FF74E64F61D]
	(No symbol) [0x00007FF74E63EB79]
	BaseThreadInitThunk [0x00007FFD25297374+20]
	RtlUserThreadStart [0x00007FFD26F9CC91+33]

Error retrieving input value: Message: no such window: target window alre

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=130.0.6723.59)
Stacktrace:
	GetHandleVerifier [0x00007FF74E6E3AB5+28005]
	(No symbol) [0x00007FF74E6483B0]
	(No symbol) [0x00007FF74E4E580A]
	(No symbol) [0x00007FF74E4BFA85]
	(No symbol) [0x00007FF74E562AD7]
	(No symbol) [0x00007FF74E57B1B1]
	(No symbol) [0x00007FF74E55B7E3]
	(No symbol) [0x00007FF74E5275C8]
	(No symbol) [0x00007FF74E528731]
	GetHandleVerifier [0x00007FF74E9D643D+3118829]
	GetHandleVerifier [0x00007FF74EA26C90+3448640]
	GetHandleVerifier [0x00007FF74EA1CF0D+3408317]
	GetHandleVerifier [0x00007FF74E7AA40B+841403]
	(No symbol) [0x00007FF74E65340F]
	(No symbol) [0x00007FF74E64F484]
	(No symbol) [0x00007FF74E64F61D]
	(No symbol) [0x00007FF74E63EB79]
	BaseThreadInitThunk [0x00007FFD25297374+20]
	RtlUserThreadStart [0x00007FFD26F9CC91+33]
